This file select for border counties that have experienced difference in policy at some time point with their neighboring counties in another state.

Cases were chosen where for both states all border counties are under the same covid policy(eg state1: all border counties have masking mandates; state2: all border counties does not have masking mandate). Which left us with a total of 82 borders for masking mandates. 

Input files: Policies.csv, county_adjacency.csv(retrieved from US census https://www.census.gov/)
Output file: mask_test.csv, which will be used as input for t-testing

In [219]:
import pandas as pd

In [238]:
# process county_adjacency.csv, gen intermediate output file border_county_out.xlsx
neighbor_df = pd.read_csv('county_adjacency.csv', dtype=object)
neighbor_df[['countyname','county_state']] = neighbor_df['countyname'].str.split(', ',expand=True)
neighbor_df[['neighborname','neighbor_state']] = neighbor_df['neighborname'].str.split(', ',expand=True)
df_nei=neighbor_df.loc[neighbor_df['county_state'] != neighbor_df['neighbor_state']]
#df_nei.to_excel('border_county.xlsx')
df_nei['fipscounty']=df_nei['fipscounty']+','
df_nei['fipsneighbor']=df_nei['fipsneighbor']+','
df_border=df_nei.groupby(['county_state', 'neighbor_state']).sum()
df_border.drop(columns=['countyname','neighborname'])
df_border.to_excel('border_county_out.xlsx')


<ipython-input-238-41afdcd88643>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nei['fipscounty']=df_nei['fipscounty']+','
<ipython-input-238-41afdcd88643>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nei['fipsneighbor']=df_nei['fipsneighbor']+','


In [241]:
bc = pd.read_excel('border_county_out.xlsx', dtype=object)
bc=bc.fillna(method='ffill')
bc=bc.drop(columns=['countyname','neighborname'])


# drop the duplicates of county_state and neighbor_state_pair

# loop through each row, 
#     get value of county_state,neighbor_state;
#         check if [neighbor_state,county_state] in pairs:
#             if in, expand list 'rows_to_drop'
#             if not, expand [county_state,neighbor_state] to list of pairs
#         expand to list of pairs
rows_to_drop=[]
pairs=[]
i=0
for row in bc.itertuples():
    c_state=row.county_state
    n_state=row.neighbor_state
    idex=i
    if [n_state, c_state] in pairs:
        rows_to_drop.append(i)
    else:
        pairs.append([c_state, n_state])
    #print(c_state,n_state,idex)
    
    i+=1
bc=bc.drop(bc.index[rows_to_drop]).reset_index().drop(columns=['index'])

# drop duplicates in fipscounty and fips neighbor
def split_str(input_str):
    a=list(set(input_str.split(',')[:-1]))
    return(a)
bc['fipscounty']=bc['fipscounty'].apply(split_str)
bc['fipsneighbor']=bc['fipsneighbor'].apply(split_str)
bc.head()

,county_state,neighbor_state,fipscounty,fipsneighbor
0,AL,FL,"[01061, 01053, 01039, 01003, 01069]","[12033, 12131, 12113, 12063, 12059, 12091]"
1,AL,GA,"[01071, 01005, 01019, 01017, 01113, 01049, 010...","[13149, 13295, 13253, 13083, 13061, 13259, 131..."
2,AL,MS,"[01023, 01093, 01077, 01059, 01075, 01107, 011...","[28153, 28069, 28039, 28095, 28141, 28087, 280..."
3,AL,TN,"[01083, 01077, 01089, 01071]","[47055, 47099, 47071, 47103, 47181, 47051, 47115]"
4,AR,LA,"[05017, 05139, 05091, 05027, 05003, 05073]","[22067, 22027, 22111, 22123, 22119, 22015, 220..."


In [229]:
policy = pd.read_csv('Policies.csv')
policy=policy.drop(columns=['Unnamed: 0'])
policy['CountyFIPS']=policy['CountyFIPS'].apply(lambda x: '{0:0>5}'.format(x))
policy.head()

,State,CountyFIPS,County,week,Mask_Mandates,SAH,BusClose
0,Alabama,01001,Autauga,15,1,7,4
1,Alabama,01001,Autauga,16,1,7,4
2,Alabama,01001,Autauga,17,1,7,4
3,Alabama,01001,Autauga,18,1,7,4
4,Alabama,01001,Autauga,19,1,2,4


### Create DF for masking mandate border states

#### change SAH into binary categories if running for SAH
(1) order and no order(1->1, 2,3,4,5,6,7->2)

(2) mandatory and non mandatory (1,2 ->1, 3,4,5,6,7->2)


In [242]:
# # example code(run one of the section in this file to produce result for SAH)
# # (1) order and no order
# policy2=policy.copy()
# sah2 = {1: 1,2: 2,3: 2,4: 2,5: 2,6: 2,7: 2} 
# policy=policy2.replace({
#     'SAH': sah2
# })

# # (2) mandatory and non mandatory (1,2 ->1, 3,4,5,6,7->2)
# policy2=policy.copy()
# sah2 = {1: 1,2: 1,3: 2,4: 2,5: 2,6: 2,7: 2} 
# policy=policy2.replace({
#     'SAH': sah2
# })



In [231]:
weeks=list(set(policy['week']))
to_test=[]
week_diff=[]

master_county_avg_list=[]
master_neighbor_avg_list=[]

for row in bc.itertuples():
    c_state=row.county_state
    ls1=row.fipscounty
    n_state=row.neighbor_state
    ls2=row.fipsneighbor
    
    county_mask_avg_ls=[]
    neighbor_mask_avg_ls=[]

    for week in weeks:
        #print('this is week', week)
        county_mask_sum=0
        neighbor_mask_sum=0
        wk=policy.loc[policy['week'] == week]
        for c_fip in ls1:
            each_county_mask=(wk.loc[wk['CountyFIPS'] == c_fip, 'Mask_Mandates'].sum())
            county_mask_sum+=each_county_mask
            county_mask_avg=county_mask_sum/len(ls1)

        #print('county_mask_avg is',county_mask_avg)
        county_mask_avg_ls.append(county_mask_avg)

        for n_fip in ls2:
            each_neighbor_mask=(wk.loc[wk['CountyFIPS'] == n_fip, 'Mask_Mandates'].sum())
            neighbor_mask_sum+=each_neighbor_mask
            neighbor_mask_avg=neighbor_mask_sum/len(ls2)

        #print('neighbor_mask_avg is',neighbor_mask_avg)
        neighbor_mask_avg_ls.append(neighbor_mask_avg)

        if county_mask_avg==1 and neighbor_mask_avg==2:
            #print('y')
            if [c_state,n_state,'neighbor_county_masked'] not in to_test:
                to_test.append([c_state,n_state,'neighbor_county_masked'])
                week_diff.append(week)
        elif county_mask_avg==2 and neighbor_mask_avg==1:
            #print('y')
            if [c_state,n_state,'county_masked'] not in to_test:
                to_test.append([c_state,n_state,'county_masked'])
                week_diff.append(week)
    master_county_avg_list.append(county_mask_avg_ls)
    master_neighbor_avg_list.append(neighbor_mask_avg_ls)

In [232]:
mask_border_county=pd.DataFrame(to_test,columns=['county','neighbor_county','which_county_masked'])
mask_border_county['week_diff']=week_diff
mask_border_county

bc['county_mask_index_wk15_53']=master_county_avg_list
bc['neighbor_county_mask_index_wk15_53']=master_neighbor_avg_list

mask_out=pd.merge(bc, mask_border_county,  how='inner', 
         left_on=['county_state','neighbor_state'], right_on = ['county','neighbor_county']).drop(columns=['county','neighbor_county'])


### Output file discription
county_state: state of county

neighbor_state: state of neighbor

fipscounty&fipsneighbor: fips code of counties in this [county_state, neighbor_state] match that are on border

county_mask_index_wk15-53&neighbor_county_mask_index_wk15-53: a list average mask index of border counties for week 15-53

which_county_masked: Specifies which state has the public masking mandate

week_diff: the week where their start to be a difference in masking mandate




In [235]:
# drop cases where not all border counties hace the same policy
drop_ls=[]

for row in mask_out.itertuples():
    c_score=row.county_mask_index_wk15_53
    n_score=row.neighbor_county_mask_index_wk15_53
    #mask_state=row.which_county_masked
    drop='no'
    row_index=getattr(row, 'Index')

    for each in c_score:
        if 1<each<2:
            drop='yes'
            
    for each in n_score:
        if 1<each<2:
            drop='yes'
            
    drop_ls.append(drop)
drop_ls
mask_out['drop']=drop_ls
mask_out=mask_out.loc[mask_out['drop']=='no']
mask_out=mask_out.drop(columns=['drop'])
mask_out
mask_out.to_csv('mask_test.csv')

,county_state,neighbor_state,fipscounty,fipsneighbor,county_mask_index_wk15_53,neighbor_county_mask_index_wk15_53,which_county_masked,week_diff
0,AL,FL,"[01061, 01053, 01039, 01003, 01069]","[12033, 12131, 12113, 12063, 12059, 12091]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",county_masked,29
1,AL,GA,"[01071, 01005, 01019, 01017, 01113, 01049, 010...","[13149, 13295, 13253, 13083, 13061, 13259, 131...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",county_masked,29
3,AL,TN,"[01083, 01077, 01089, 01071]","[47055, 47099, 47071, 47103, 47181, 47051, 47115]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",county_masked,29
4,AR,LA,"[05017, 05139, 05091, 05027, 05003, 05073]","[22067, 22027, 22111, 22123, 22119, 22015, 220...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",neighbor_county_masked,29
5,AR,MO,"[05121, 05089, 05093, 05055, 05007, 05031, 050...","[29155, 29209, 29023, 29119, 29213, 29069, 291...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",county_masked,30
...,...,...,...,...,...,...,...,...
91,OK,TX,"[40007, 40141, 40067, 40065, 40045, 40085, 400...","[48337, 48195, 48357, 48421, 48075, 48485, 480...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",neighbor_county_masked,27
92,OR,WA,"[41051, 41063, 41065, 41009, 41007, 41059, 410...","[53005, 53069, 53015, 53049, 53059, 53011, 530...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",neighbor_county_masked,26
93,PA,WV,"[42125, 42007, 42059, 42051]","[54051, 54029, 54009, 54061, 54077, 54103, 54069]","[1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",county_masked,16
94,TN,VA,"[47163, 47067, 47073, 47025, 47091]","[51169, 51520, 51077, 51105, 51191]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, ...",neighbor_county_masked,22
